In [19]:
using Pkg
Pkg.add("FIB")
Pkg.add("Parameters")
Pkg.add("StaticArrays")


 Resolving package versions...
  Updating `~/.julia/environments/v1.0/Project.toml`
 [no changes]
  Updating `~/.julia/environments/v1.0/Manifest.toml`
 [no changes]


In [23]:
using POMDPs
using Random 
using Parameters
using StaticArrays
using Distributions
using FIB
using MCTS

In [24]:
const Vec2 = SVector{2, Float64}
const Vec3 = SVector{3, Float64}

SArray{Tuple{3},Float64,1,3}

In [31]:
struct mystate
    uavPose::Vec3
    uavHeading::Float64 # radius
    targetPose::Vec2
end

struct myaction
    xy_speed::Float64 # m/s
    z_speed::Float64
    angle::Float64 # radius
end

@with_kw mutable struct UAVMDP <: MDP{mystate, myaction}

    target_velocity::Vec2 = SVector(0.5,0.5)
    target_std::Float64 = 0.1

    r_outScene::Float64 = -70
    r_action::Float64 = -10
    r_reach::Float64 = 100

    discount::Float64 = 0.95
    
    boundary::Int = 100
    landing_r::Int = 1
end

UAVMDP

In [33]:
# tolenrance can be changed
isterminal(p::UAVMDP,s::mystate) = sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2)< tol || norm(s.uavPose)>p.boundary

isterminal (generic function with 1 method)

In [34]:
function POMDPs.generate_s(p::UAVMDP, s::mystate, a::myaction)
    # calculate target state
    target_dt_distance = p.target_velocity
    sensor_noise = rand(Normal(0, p.target_std), 2)
    curr_targ = s.targetPose + target_dt_distance + sensor_noise # next_target_pos(p, s.targetPose)
    # calculate UAV state
    curr_angle = s.uavHeading + a.angle
    xy_dt_distance = a.xy_speed*SVector(cos(curr_angle), sin(curr_angle)) # careful
    z_dt_distance = a.z_speed
    xyz_dt_distance = SVector(xy_dt_distance[1], xy_dt_distance[2], z_dt_distance)
    curr_pos = s.uavPose + xyz_dt_distance
    return mystate(curr_pos, curr_angle, curr_targ)
end

In [35]:
function myreward(p::UAVMDP, s::mystate, a::myaction)
    
    distance_to_target = sqrt((s.uavPose[1]-s.targetPose[1])^2 + (s.uavPose[2]-s.targetPose[2])^2 + s.uavPose[3]^2)
    
    reward = p.r_action + 1/(distance_to_target+1)
#     out scene punish 
    if s.uavPose[1]>p.boundary || s.uavPose[2]>p.boundary
        reward = reward + p.r_outScene
        p.mission_end = true 
    end
    
    if distance_to_target < p.landing_r
        reward = reward + p.r_reach
        p.mission_end = true
    end
    return reward
end

myreward (generic function with 1 method)

In [36]:
function POMDPs.actions(p::UAVMDP)
    action_space = [myaction(i, j, k) for i=-5.0:5.0,j=-pi:pi,k=-5.0:5.0]
    return action_space
end

In [37]:
function POMDPs.generate_sr(p::UAVMDP, s::mystate, a::myaction, rng::AbstractRNG)
    sp = generate_s(p, s, a)
    r = myreward(p, s, a)
    return sp, r
end

In [38]:
POMDPs.initialstate_distribution(p::UAVMDP) = Determinstic(mystate(SVector(10,10,10),0,SVector(0,0)))
POMDPs.discount(p::UAVMDP) = p.discount
solver = MCTSSolver(n_iterations=50, depth=20, exploration_constant=5.0)
UAV = UAVMDP()

UAVMDP
  target_velocity: SArray{Tuple{2},Float64,1,2}
  target_std: Float64 0.1
  r_outScene: Float64 -70.0
  r_action: Float64 -10.0
  r_reach: Float64 100.0
  discount: Float64 0.95
  boundary: Int64 100
  landing_r: Int64 1


In [39]:
policy = solve(solver, UAV)

MCTSPlanner{UAVMDP,mystate,myaction,MCTS.SolvedRolloutEstimator{POMDPPolicies.RandomPolicy{MersenneTwister,UAVMDP,BeliefUpdaters.NothingUpdater},MersenneTwister},MersenneTwister}(MCTSSolver(50, 20, 5.0, MersenneTwister(UInt32[0xac3a4c65, 0xcb160085, 0xfde8247d, 0x5795a2b2], Random.DSFMT.DSFMT_state(Int32[1881025636, 1072710410, -630953235, 1072707728, 1766195831, 1072966342, -2086708029, 1073016965, -685799300, 1073421160  …  -2092019959, 1073030175, -337879850, 1073187049, 846234779, 1115230528, -68748774, 968711016, 382, 0]), [1.61562, 1.5209, 1.63043, 1.93054, 1.00505, 1.6204, 1.57115, 1.88707, 1.46959, 1.26853  …  1.25298, 1.25967, 1.23643, 1.83813, 1.94244, 1.35332, 1.87323, 1.10955, 1.32132, 1.47093], UInt128[0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x00000000000000000000000000000000, 0x000000000000000000000

In [40]:
a = action(policy,mystate(SVector(10,10,10),0,SVector(0,0)))

myaction(1.0, 0.8584073464102069, -5.0)